## Settings

In [38]:
%matplotlib inline
import os
os.chdir('U:\\GitHubClones\\SALTSeniorDesign')
os.getcwd()
import Excel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import re
from datetime import datetime

In [39]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Path Setup For the File System

In [40]:
path = Excel.directory + "\\" + Excel.shelf + "\\" + Excel.deal + "\\" + Excel.dfTapes[Excel.deal][0]
print(path)

U:\AutoLoanTapes\Santander Drive Auto Receivables Trust\SDART 2017-1\2017\01 January\2017.02.15_Santander Drive Auto Receivables Trust 2017-1 Data Tape.csv


*Reading in CSV File*

In [41]:
df = pd.read_csv(path, memory_map=True, low_memory=False)
total = len(df.index)
print(total)
df['reportingPeriodActualEndBalanceAmount'].sum()

66797


1224015127.290021

### Snap Shot of the Data

In [6]:
df.head()

,assetTypeNumber,assetNumber,reportingPeriodBeginningDate,reportingPeriodEndingDate,originatorName,originationDate,originalLoanAmount,originalLoanTerm,loanMaturityDate,originalInterestRatePercentage,interestCalculationTypeCode,originalInterestRateTypeCode,originalInterestOnlyTermNumber,originalFirstPaymentDate,underwritingIndicator,gracePeriodNumber,paymentTypeCode,subvented,vehicleManufacturerName,vehicleModelName,vehicleNewUsedCode,vehicleModelYear,vehicleTypeCode,vehicleValueAmount,vehicleValueSourceCode,obligorCreditScoreType,obligorCreditScore,obligorIncomeVerificationLevelCode,obligorEmploymentVerificationCode,coObligorIndicator,paymentToIncomePercentage,obligorGeographicLocation,assetAddedIndicator,remainingTermToMaturityNumber,reportingPeriodModificationIndicator,servicingAdvanceMethodCode,reportingPeriodBeginningLoanBalanceAmount,nextReportingPeriodPaymentAmountDue,reportingPeriodInterestRatePercentage,nextInterestRatePercentage,servicingFeePercentage,servicingFlatFeeAmount,otherServicerFeeRetainedByServicer,otherAssessedUncollectedServicerFeeAmount,scheduledInterestAmount,scheduledPrincipalAmount,otherPrincipalAdjustmentAmount,reportingPeriodActualEndBalanceAmount,reportingPeriodScheduledPaymentAmount,totalActualAmountPaid,actualInterestCollectedAmount,actualPrincipalCollectedAmount,actualOtherCollectedAmount,servicerAdvancedAmount,interestPaidThroughDate,zeroBalanceEffectiveDate,zeroBalanceCode,currentDelinquencyStatus,primaryLoanServicerName,mostRecentServicingTransferReceivedDate,assetSubjectDemandIndicator,assetSubjectDemandStatusCode,repurchaseAmount,DemandResolutionDate,repurchaserName,repurchaseReplacementReasonCode,chargedoffPrincipalAmount,recoveredAmount,modificationTypeCode,paymentExtendedNumber,repossessedIndicator,repossessedProceedsAmount
0,SC,"=""6345119""",01-01-2017,01-31-2017,SC,10/2013,23260.03,60,01/2019,0.1194,1,1,-,12/2013,True,2,2,0,CHEVROLET TRUCK,Traverse,2,2010,3,17589.00,98,Bureau,671,3,2,False,0.187788,AZ,-,25,False,1,11977.51,512.31,0.1194,0.1194,0.0,-,-,50.59,103.1772,413.5228,0.0,11607.02,516.70,519.25,148.76,370.49,0.0,-,01-27-2017,-,-,0,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-
1,SC,"=""634535""",01-01-2017,01-31-2017,SC,03/2012,12031.00,72,03/2018,0.1800,1,1,-,04/2012,True,1,2,0,MAZDA,Mazda3,2,2010,1,13575.00,98,Bureau,-,2,2,False,0.069108,TX,-,15,False,1,3658.90,284.44,0.1800,0.1800,0.0,-,-,0.00,53.2969,221.1031,0.0,3439.80,274.40,275.00,55.90,219.10,0.0,-,01-24-2017,-,-,0,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-
2,SC,"=""6345360""",01-01-2017,01-31-2017,SC,11/2013,21332.31,72,11/2019,0.1810,1,1,-,12/2013,True,1,2,0,DODGE TRUCK,Grand Caravan,2,2010,1,15209.00,3,Bureau,560,2,2,False,0.109852,WA,-,35,False,1,14132.08,1317.82,0.1810,0.1810,0.0,-,-,0.00,196.7817,290.9683,0.0,13850.49,487.75,491.37,209.78,281.59,0.0,-,01-07-2017,-,-,0,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-
3,SC,"=""6345391""",01-01-2017,01-31-2017,SC,11/2013,24544.23,72,11/2019,0.0855,1,1,-,12/2013,True,1,2,0,TOYOTA,Sienna,2,2006,1,16550.00,98,Bureau,712,2,2,False,0.007974,FL,-,35,False,1,13122.36,538.38,0.0855,0.0855,0.0,-,-,0.00,96.1377,340.8223,0.0,13122.36,436.96,0.00,0.00,0.00,0.0,-,12-30-2016,-,-,0,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-
4,SC,"=""634551""",01-01-2017,01-31-2017,SC,03/2012,23589.10,72,09/2018,0.1710,1,1,-,04/2012,True,1,2,0,CHEVROLET,Sonic,1,2012,1,16607.31,98,Bureau,631,3,3,False,0.074936,IL,-,21,False,1,10510.59,1210.41,0.1710,0.1710,0.0,-,-,78.09,97.8123,428.2677,0.0,10510.59,526.08,0.00,0.00,0.00,0.0,-,10-07-2016,-,-,3,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-


In [6]:
list(df.columns)

['assetTypeNumber',
 'assetNumber',
 'reportingPeriodBeginningDate',
 'reportingPeriodEndingDate',
 'originatorName',
 'originationDate',
 'originalLoanAmount',
 'originalLoanTerm',
 'loanMaturityDate',
 'originalInterestRatePercentage',
 'interestCalculationTypeCode',
 'originalInterestRateTypeCode',
 'originalInterestOnlyTermNumber',
 'originalFirstPaymentDate',
 'underwritingIndicator',
 'gracePeriodNumber',
 'paymentTypeCode',
 'subvented',
 'vehicleManufacturerName',
 'vehicleModelName',
 'vehicleNewUsedCode',
 'vehicleModelYear',
 'vehicleTypeCode',
 'vehicleValueAmount',
 'vehicleValueSourceCode',
 'obligorCreditScoreType',
 'obligorCreditScore',
 'obligorIncomeVerificationLevelCode',
 'obligorEmploymentVerificationCode',
 'coObligorIndicator',
 'paymentToIncomePercentage',
 'obligorGeographicLocation',
 'assetAddedIndicator',
 'remainingTermToMaturityNumber',
 'reportingPeriodModificationIndicator',
 'servicingAdvanceMethodCode',
 'reportingPeriodBeginningLoanBalanceAmount',
 '

### Feature Calculation Steps

In [42]:
WA = lambda x: np.average(x, axis=0, weights = df.loc[x.index, 'reportingPeriodActualEndBalanceAmount'])

In [49]:
df['LTV'] = (df['originalLoanAmount']/df['vehicleValueAmount']) #Loan to value ratio
df['pctDeal'] = (df['reportingPeriodActualEndBalanceAmount']/(df['reportingPeriodActualEndBalanceAmount'].sum()))*100 #Individual Loan as a Percent of Current month Balance

print(df['pctDeal'].head())
df['LTV'].head()

0    0.000948
1    0.000281
2    0.001132
3    0.001072
4    0.000859
Name: pctDeal, dtype: float64


0    1.322419
1    0.886262
2    1.402611
3    1.483035
4    1.420405
Name: LTV, dtype: float64

array(['-', '4', '1', '3'], dtype=object)

## Selecting and Finding Data Characteristics

In [11]:
df['originalInterestRateTypeCode'].unique() #all of the loans are fixed rate loans
df['subvented'].unique() #1=rate, 2=cash, 0= none, 98=other
df['vehicleTypeCode'].unique() #1=car 2=truck 3=suv
df['vehicleNewUsedCode'].unique() #1=new 2=used
df['vehicleManufacturerName'].unique()
df['vehicleModelName'].unique()
df['vehicleValueAmount'].dtype #float
df['vehicleValueSourceCode'].unique() #98=other 3=KBB
df['obligorCreditScoreType'].unique() #only Bureau
df['obligorCreditScore'].dtype #float
df['obligorIncomeVerificationLevelCode'].unique() #3=Stated, verified but not to “level 4” or “level 5” 2=Stated, not verified
df['obligorEmploymentVerificationCode'].unique() # 1=not stated 2=stated not verified 3=verifited by 3rd party
df['paymentToIncomePercentage'].dtype #float
df['obligorGeographicLocation'].unique()
#df['assetAddedIndicator'].unique()
df['remainingTermToMaturityNumber'].dtype #int
df['reportingPeriodModificationIndicator'].unique()
df['servicingAdvanceMethodCode'].unique()
df['reportingPeriodBeginningLoanBalanceAmount'].dtype #float
df['totalActualAmountPaid'].dtype #float
df['actualInterestCollectedAmount'].dtype #float
df['actualPrincipalCollectedAmount'].dtype #float
df['zeroBalanceCode'].unique()
df['zeroBalanceEffectiveDate'].unique() #date at which the loan is considered dead
df['currentDelinquencyStatus'].unique() # How many days delinquent
df['assetSubjectDemandIndicator'].unique()
df['assetSubjectDemandStatusCode'].unique() #Asset has been repurchased replaced ect
df['repurchaseAmount'].unique()
df['repurchaseReplacementReasonCode'].unique() #reason why the car has thrown a zero balance code
df['chargedoffPrincipalAmount'].dtype #float
df['recoveredAmount'].dtype #float
df['modificationTypeCode'].unique() #what part of the loan has been modified
df['paymentExtendedNumber'].unique()
df['repossessedIndicator'].unique()
df['repossessedProceedsAmount'].dtype #object
df['LTV'].dtype #float
df['pctDeal'].dtype #float
df['coObligorIndicator'].value_counts()

False    43209
True     20238
Name: coObligorIndicator, dtype: int64

In [54]:
df['LTV'] = pd.to_numeric(df['LTV'], errors='coerce')

### Data Cleaning Steps

In [55]:
noScore = df[df.obligorCreditScore.isin(['-'])]
noScore['obligorCreditScore'].describe()

count     9594
unique       1
top          -
freq      9594
Name: obligorCreditScore, dtype: object

In [56]:
print(df['obligorCreditScore'].describe())
df['obligorCreditScore'] = pd.to_numeric(df['obligorCreditScore'], errors='coerce')
print(df['obligorCreditScore'].describe())

count     66797
unique      483
top           -
freq       9594
Name: obligorCreditScore, dtype: object
count    57203.000000
mean       603.404542
std         64.530552
min        385.000000
25%        562.000000
50%        600.000000
75%        637.000000
max        900.000000
Name: obligorCreditScore, dtype: float64


In [57]:
df['reportingPeriodActualEndBalanceAmount'].describe()

count    66797.000000
mean     18324.402702
std       9202.795412
min        505.800000
25%      11815.000000
50%      16428.580000
75%      23109.430000
max      72363.060000
Name: reportingPeriodActualEndBalanceAmount, dtype: float64

In [ ]:
#df['obligorCreditScore'].plot.hist(bins=100).set_xlabel('SDART 2017-1 Credit Score')
#df['LTV'].plot.hist(bins=100).set_xlabel('SDART 2017-1 LTV')
#df['originalInterestRatePercentage'].plot.hist(bins=50).set_xlabel('SDART 2017-1 APR')
#df['paymentToIncomePercentage'].plot.hist(bins=100).set_xlabel('SDART 2017-1 PTI')

### Bin Selection for Strat Tables

In [58]:
FicoBins = [-1, 440, 500, 560, 620, 680, 740, 800, 1000]
#APRBins = [0, .02 ,.04, .06, .08, .1, .12, .14, .16, 1000]
APRBins = [0, .04, .08, .12, .16, .20, .24, .28, 1000]
LTVBins = [-1, .7, .8, .9, 1, 1.1, 1.2, 1.3, 1.4, 1000]
PTIBins = [-1, .02, .04, .06, .08, .1, .12, .14, .16, 1000]

In [59]:
FicoGroup = ['[-1, 440)', '[440, 500)', '[500, 560)', '[560, 620)', '[620, 680)', '[680, 740)', '[740, 800)', '[800, 1000)']
#APRGroup = ['[0.0, 0.02)', '[0.02, 0.04)', '[0.04, 0.06)', '[0.06, 0.08)', '[0.08, 0.1)', '[0.1, 0.12)', '[0.12, 0.14)', '[0.14, 0.16)', '[0.16, 1000.0)']
APRGroup = ['[0.0, 0.04)', '[0.04, 0.08)', '[0.08, 0.12)', '[0.12, 0.16)', '[0.16, 0.20)', '[0.20, 0.24)', '[0.24, 0.28)', '[0.28, 1000)']
PTIGroup = ['[-1.0, 0.02)', '[0.02, 0.04)', '[0.04, 0.06)', '[0.06, 0.08)', '[0.08, 0.1)', '[0.1, 0.12)', '[0.12, 0.14)', '[0.14, 0.16)', '[0.16, 1000.0)']

LTVGroup = ['[-1.0, 0.7)', '[0.7, 0.8)', '[0.8, 0.9)', '[0.9, 1.0)', '[1.0, 1.1)', '[1.1, 1.2)', '[1.2, 1.3)', '[1.3, 1.4)', '[1.4, 1000.0)']

### Cutting the Data Based on Bin Size

In [60]:
df['Fico Group'] = pd.cut(df['obligorCreditScore'], FicoBins, right=True, labels=FicoGroup)
#df['APR Group'] = pd.cut(df['originalInterestRatePercentage'], APRBins, labels=APRGroup)
df['APR Group'] = pd.cut(df['originalInterestRatePercentage'], APRBins, labels=APRGroup)
df['PTI Group'] = pd.cut(df['paymentToIncomePercentage'], PTIBins, labels=PTIGroup)
df['LTV Group']= pd.cut(df['LTV'], LTVBins, labels=LTVGroup)

In [61]:
df['LTV Group']

0           [1.3, 1.4)
1           [0.8, 0.9)
2        [1.4, 1000.0)
3        [1.4, 1000.0)
4        [1.4, 1000.0)
5           [1.2, 1.3)
6           [1.1, 1.2)
7           [1.3, 1.4)
8           [1.2, 1.3)
9           [1.1, 1.2)
10       [1.4, 1000.0)
11          [1.3, 1.4)
12          [1.3, 1.4)
13          [1.1, 1.2)
14          [1.2, 1.3)
15          [1.1, 1.2)
16       [1.4, 1000.0)
17          [1.1, 1.2)
18          [0.9, 1.0)
19          [1.0, 1.1)
20       [1.4, 1000.0)
21          [1.2, 1.3)
22          [1.2, 1.3)
23          [0.8, 0.9)
24          [1.0, 1.1)
25          [1.1, 1.2)
26       [1.4, 1000.0)
27          [1.3, 1.4)
28          [1.1, 1.2)
29          [1.0, 1.1)
30          [1.3, 1.4)
31          [1.1, 1.2)
32          [1.1, 1.2)
33          [1.0, 1.1)
34          [1.2, 1.3)
35          [1.2, 1.3)
36          [1.0, 1.1)
37          [1.2, 1.3)
38          [0.7, 0.8)
39          [1.3, 1.4)
40          [1.2, 1.3)
41          [1.3, 1.4)
42          [0.7, 0.8)
43         

#### Added No Score Category

In [23]:
df['Fico Group'] = df['Fico Group'].cat.add_categories('No Score').fillna('No Score')

In [24]:
df['Fico Group'].head()

0    [620, 680)
1      No Score
2    [500, 560)
3    [680, 740)
4    [620, 680)
Name: Fico Group, dtype: category
Categories (9, object): [[-1, 440) < [440, 500) < [500, 560) < [560, 620) ... [680, 740) < [740, 800) < [800, 1000) < No Score]

### Ordered Dictionary for Loan Characteristics in Seclected Bins

In [62]:
a = OrderedDict([
    ('assetNumber',['count']),
    ('reportingPeriodActualEndBalanceAmount', ['sum','mean']),
    ('pctDeal', ['sum']),
    ('originalLoanTerm',[WA]),
    ('originalInterestRatePercentage',[WA])
])

In [57]:
df[['assetNumber', 'pctDeal','reportingPeriodActualEndBalanceAmount','originalLoanTerm','originalInterestRatePercentage']].groupby([df['Fico Group'], df['APR Group4']]).agg(a)

assetNumber reportingPeriodActualEndBalanceAmount                  pctDeal originalLoanTerm originalInterestRatePercentage
                               count                                   sum          mean        sum         <lambda>                       <lambda>
Fico Group  APR Group4                                                                                                                             
[-1, 440)   [0.08, 0.12)           2                          3.707164e+04  18535.820000   0.003808        74.113460                       0.117443
            [0.12, 0.16)           9                          1.168860e+05  12987.332222   0.012006        70.663388                       0.143945
            [0.16, 0.20)          40                          5.216711e+05  13041.777250   0.053582        71.150105                       0.181397
            [0.20, 0.24)          48                          5.230345e+05  10896.552292   0.053722        70.956498                       0.217354
            [0.24, 0.28)          27                          2.796532e+05  10357.525556   0.028724        68.795704                       0.252990
[440, 500)  [0.0, 0.04)            1                          1.127083e+04  11270.830000   0.001158        72.000000                       0.039000
            [0.04, 0.08)          14                          2.154939e+05  15392.423571   0.022134        72.619339                       0.068430
            [0.08, 0.12)          61                          1.019707e+06  16716.515902   0.104736        72.079275                       0.103020
            [0.12, 0.16)         205                          3.298666e+06  16091.052488   0.338811        71.569338                       0.145090
            [0.16, 0.20)         757                          9.883595e+06  13056.267952   1.015159        71.232997                       0.180749
            [0.20, 0.24)         640                          7.294430e+06  11397.547547   0.749222        70.740058                       0.217108
            [0.24, 0.28)         343                          3.413102e+06   9950.735802   0.350565        70.250083                       0.255185
            [0.28, 1000)           1                          1.479680e+03   1479.680000   0.000152        60.000000                       0.291700
[500, 560)  [0.0, 0.04)            9                          1.118481e+05  12427.570000   0.011488        71.128167                       0.033289
            [0.04, 0.08)         175                          3.308494e+06  18905.677943   0.339820        71.628673                       0.069268
            [0.08, 0.12)         838                          1.695862e+07  20237.016551   1.741846        71.684470                       0.103839
            [0.12, 0.16)        1992                          3.588122e+07  18012.662610   3.685416        71.889690                       0.144105
            [0.16, 0.20)        4405                          6.625491e+07  15040.841750   6.805143        71.324167                       0.178814
            [0.20, 0.24)        2278                          2.733928e+07  12001.439662   2.808059        70.264688                       0.216580
            [0.24, 0.28)         696                          7.209812e+06  10358.925302   0.740531        69.976394                       0.254603
            [0.28, 1000)           1                          1.110304e+04  11103.040000   0.001140        72.000000                       0.290200
[560, 620)  [0.0, 0.04)           99                          2.107045e+06  21283.281414   0.216418        71.363852                       0.031867
            [0.04, 0.08)         760                          1.737881e+07  22866.855079   1.785004        72.494772                       0.067985
            [0.08, 0.12)        2921                          6.133934e+07  20999.430924   6.300258        72.159379                       0.102630
            [0.12, 0.16)     

In [63]:
stratTable = df[['assetNumber', 'pctDeal','reportingPeriodActualEndBalanceAmount','originalLoanTerm', 'originalInterestRatePercentage']].groupby([df['LTV Group']]).agg(a)
lis = ['Fico Group', 'APR Group', 'LTV Group', 'PTI Group']
#for i in range(0, len(lis)):
#    stratTable = df[['assetNumber', 'pctDeal','reportingPeriodActualEndBalanceAmount','originalLoanTerm', 'originalInterestRatePercentage']].groupby([df[lis[i]]]).agg(a)

stratTable.columns = ['Loan Count', 'Sum of Remaining Balance', 'Average Remaining Balance', 'Sum of Total Loan Balance Percentage', 'Weighted Average Original Term','Weighted Average APR']
stratTable.to_clipboard()

### Formatting Rules

In [ ]:
("%.2f%%" % df['LTV'])